Nathaniel's Branch

In [4]:
# Initalize GPU
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/CAIS_Sentiment140_Project/
!unzip trainingandtestdata.zip
%ls

/content/drive/MyDrive/CAIS_Sentiment140_Project
Archive:  trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
testdata.manual.2009.06.14.csv             trainingandtestdata.zip
training.1600000.processed.noemoticon.csv


In [7]:
import pandas as pd
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text' ])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 1,600,000



In [8]:
# Display 10 random rows from the data.
df.sample(10)

,target,ids,date,flag,user,text
1242423,4,1994229862,Mon Jun 01 11:36:50 PDT 2009,NO_QUERY,Solar75,tweet tweet...tweet tweet....guitarra reggae ...
379888,0,2052444181,Sat Jun 06 00:45:03 PDT 2009,NO_QUERY,ilsegaboyil,@Grissay kinda i shoulda seen u instead
463153,0,2174855101,Sun Jun 14 23:28:31 PDT 2009,NO_QUERY,SharonSame,I love him but he hurt me so much I don't know...
544864,0,2201244122,Tue Jun 16 20:00:23 PDT 2009,NO_QUERY,claytondarogers,Watchin stand by me.....tonight was great love...
131895,0,1835631894,Mon May 18 06:25:23 PDT 2009,NO_QUERY,jordenamber,@rosie_rage hahahaha! thats abit harsh! im sur...
140913,0,1881142747,Fri May 22 03:44:36 PDT 2009,NO_QUERY,mizz_dgaf,"@taylorswift13 I really wish I could be there,..."
85450,0,1753847523,Sun May 10 03:45:45 PDT 2009,NO_QUERY,xo_mcflyandjb,@RaraACTIVE not with me and Rachel though.
1045236,4,1957530219,Fri May 29 00:47:31 PDT 2009,NO_QUERY,ronaldhiggins,@Special__K__ as if it was all for Joe .. uh ...
1126568,4,1975042684,Sat May 30 14:19:37 PDT 2009,NO_QUERY,yizzy920,note to self: Cayenne Pepper is mucho hot
1207729,4,1988737263,Sun May 31 23:13:37 PDT 2009,NO_QUERY,zashi0617,Looking for Mrs Quiros


In [9]:
print("0, negative sentiment")
df.loc[df.target == 0].sample(5)[['target', 'text']]

0, negative sentiment


,target,text
469000,0,said bye to my parents. teary eyed byes
397251,0,the shortest pool session ever with @JennaJack...
498999,0,Hasn't talked to his bestfriend all day
113195,0,Off to another soccer game - sun is out so tha...
248563,0,@thetricktolife Niceee! I need to go bike ridi...


In [10]:
print("4, positive sentiment")
df.loc[df.target == 4].sample(5)[['target', 'text']]

4, positive sentiment


,target,text
1588253,4,Loving the block button right about now. Don't...
1481441,4,At @Patar101's house! They be playing some lou...
994140,4,Good night Philippines! ) Gotta sleep! 'til ne...
1119999,4,Summer help - Hire a Teenager - Start your sea...
1208471,4,@frenchie28 ...all the time. Look out the wind...


In [11]:
#Extract text and target labels
texts = df.text.values
targets = df.target.values

In [12]:
!pip install transformers
import transformers
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 34.9 MB/s 
     |████████████████████████████████| 182 kB 71.2 MB/s 
     |████████████████████████████████| 7.6 MB 58.2 MB/s 
Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

#Potential To-Do = Further Scrub Data of mentions, links, and other punctuation

In [13]:
# Print the original sentence.
print(' Original: ', texts[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(texts[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

 Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Tokenized:  ['@', 'switch', '##foot', 'http', ':', '/', '/', 't', '##wi', '##tp', '##ic', '.', 'com', '/', '2', '##y', '##1', '##z', '##l', '-', 'aw', '##w', '##w', ',', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', ';', 'd']
Token IDs:  [1030, 6942, 13064, 8299, 1024, 1013, 1013, 1056, 9148, 25856, 2594, 1012, 4012, 1013, 1016, 2100, 2487, 2480, 2140, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040]


In [23]:
input_ids = []
attention_masks = []

# For every sentence...
for t in texts:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        t,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 140,           # Pad & truncate all sentences.
                        pad_to_max_length = True,          # Pad & truncate all sentences.
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt'     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
targets = torch.tensor(targets)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Token IDs: tensor([  101,  1030,  6942, 13064,  8299,  1024,  1013,  1013,  1056,  9148,
        25856,  2594,  1012,  4012,  1013,  1016,  2100,  2487,  2480,  2140,
         1011, 22091,  2860,  2860,  1010,  2008,  1005,  1055,  1037, 26352,
         5017,  1012,  2017,  2323,  2050,  2288,  2585, 12385,  1997,  2353,
         2154,  2000,  2079,  2009,  1012,  1025,  1040,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
    

In [30]:
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])
print('Attention Masks', attention_masks[0])
print("Targets", targets[0])

Original:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Token IDs: tensor([  101,  1030,  6942, 13064,  8299,  1024,  1013,  1013,  1056,  9148,
        25856,  2594,  1012,  4012,  1013,  1016,  2100,  2487,  2480,  2140,
         1011, 22091,  2860,  2860,  1010,  2008,  1005,  1055,  1037, 26352,
         5017,  1012,  2017,  2323,  2050,  2288,  2585, 12385,  1997,  2353,
         2154,  2000,  2079,  2009,  1012,  1025,  1040,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
    

In [34]:
import os
newpath = r'/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors' 
if not os.path.exists(newpath):
    os.makedirs(newpath)


torch.save(texts, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/texts.pt" )
torch.save(input_ids, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/input_ids.pt" )
torch.save(attention_masks, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/attention_masks.pt" )
torch.save(targets, f = "/content/drive/MyDrive/CAIS_Sentiment140_Project/savedtensors/targets.pt" )

#Word Embedding Step Done